# Merge Football-Data and Whoscored Image Paths

In [1]:
import pathlib
import sys
import pickle

import numpy as np
import pandas as pd

# Load the "autoreload" extension
%load_ext autoreload
# always reload modules marked with "%aimport"
%autoreload 1
# add the 'src' directory to path to import modules
src_dir = pathlib.Path().cwd().resolve().parent / 'src'
#src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)
# import my class code from the source
# %aimport src-dir.filename

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

PROJECT_DIR = pathlib.Path.cwd().resolve().parent
DATA_DIR = PROJECT_DIR / 'data'
RAW_DATA_DIR = DATA_DIR / '01-raw'
INTERIM_DATA_DIR = DATA_DIR / '02-interim' / 'football-data'
REF_DATA_DIR = DATA_DIR / 'reference'

## Select League Data

In [2]:
league = 'english-premier-league'
fn = str('football-data-' + league + '.csv')
df = pd.read_csv(INTERIM_DATA_DIR / league / fn, parse_dates=['date'], index_col=None)
df.head()

,a_corners,a_fouls,a_rcards,a_shots,a_sot,a_ycards,a,odds_awin_bet365,odds_draw_bet365,odds_hwin_bet365,odds_awin_BS,odds_draw_BS,odds_hwin_BS,odds_awin_BW,odds_draw_BW,odds_hwin_BW,n_Bb1X2,n_BbAsian,BbAsian_handicap,odds_ftgoalsu2.5_bbmean,odds_ftgoalso2.5_bbmean,odds_awin_bbmean,odds_asianaway_bbmean,odds_asianaway_bbmean.1,odds_draw_bbmean,odds_hwin_bbmean,odds_ftgoalsu2.5_bbmax,odds_ftgoalso2.5_bbmax,odds_awin_bbmax,odds_asianaway_bbmax,odds_asianhome_bbmax,odds_draw_bbmax,odds_hwin_bbmax,n_BbOU,date,a_ftgoals,h_ftgoals,odds_awin_GB,odds_draw_GB,odds_hwin_GB,h_corners,h_fouls,h_rcards,h_shots,h_sot,a_htgoals,h_htgoals,h_ycards,h,odds_awin_IW,odds_draw_IW,odds_hwin_IW,odds_awin_LB,odds_draw_LB,odds_hwin_LB,odds_awin_pinn,clodds_away_pinn,clodds_draw_pinn,clodds_hwin_pinn,odds_draw_pinn,odds_hwin_pinn,odds_awin_SB,odds_draw_SB,odds_hwin_SB,odds_awin_SJ,odds_draw_SJ,odds_hwin_SJ,odds_awin_VC,odds_draw_VC,odds_hwin_VC,odds_awin_WH,odds_draw_WH,odds_hwin_WH,season
0,6,14,0,14,7,2,Wigan,5.50,3.60,1.67,5.5,3.5,1.62,5.00,3.40,1.65,37,21.0,0.0,1.81,1.92,5.33,3.99,1.22,3.51,1.66,1.91,2.04,6.05,4.40,1.28,3.77,1.72,32,2009-08-15,2,0,5.25,3.50,1.67,4,15,0,11,5,1,0,2,Aston Villa,4.9,3.5,1.7,4.50,3.3,1.67,NaN,NaN,NaN,NaN,NaN,NaN,5.0,3.40,1.65,5.50,3.60,1.67,5.0,3.4,1.70,5.50,3.4,1.70,2009-2010
1,4,9,0,8,5,1,Man City,2.10,3.25,3.60,2.2,3.2,3.30,2.00,3.15,3.40,38,24.0,0.0,1.71,2.04,2.12,1.54,2.38,3.24,3.37,1.78,2.15,2.25,1.60,2.58,3.35,3.78,33,2009-08-15,2,0,2.10,3.25,3.40,5,12,0,17,9,1,0,2,Blackburn,2.2,3.2,3.2,2.10,3.2,3.00,NaN,NaN,NaN,NaN,NaN,NaN,2.1,3.25,3.15,2.20,3.25,3.30,2.2,3.2,3.25,2.15,3.2,3.50,2009-2010
2,7,10,0,20,13,1,Sunderland,3.25,3.25,2.25,3.3,3.2,2.20,3.15,3.15,2.15,38,23.0,0.0,1.66,2.12,3.15,2.23,1.61,3.20,2.24,1.70,2.25,3.40,2.33,1.68,3.31,2.37,33,2009-08-15,1,0,3.10,3.25,2.25,4,16,0,11,3,1,0,2,Bolton,3.2,3.2,2.2,3.00,3.2,2.10,NaN,NaN,NaN,NaN,NaN,NaN,2.9,3.20,2.25,3.10,3.25,2.30,3.1,3.2,2.25,3.20,3.2,2.30,2009-2010
3,4,15,0,7,3,2,Hull,21.00,6.50,1.17,19.0,5.5,1.20,13.00,6.00,1.17,38,22.0,0.0,2.26,1.58,16.39,12.96,1.02,6.26,1.17,2.44,1.67,23.14,17.05,1.03,7.71,1.20,34,2009-08-15,1,2,15.00,6.00,1.20,12,13,0,26,12,1,1,1,Chelsea,14.0,5.5,1.2,12.00,5.5,1.17,NaN,NaN,NaN,NaN,NaN,NaN,15.0,5.75,1.17,17.00,6.50,1.18,17.0,6.0,1.17,21.00,6.5,1.17,2009-2010
4,9,13,0,15,9,0,Arsenal,2.30,3.25,3.20,2.2,3.2,3.25,2.25,3.15,2.95,38,24.0,0.0,1.70,2.06,2.28,1.63,2.20,3.21,3.07,1.77,2.20,2.42,1.73,2.27,3.34,3.34,34,2009-08-15,6,1,2.30,3.25,3.00,4,11,0,8,5,3,0,0,Everton,2.3,3.1,3.1,2.25,3.0,2.88,NaN,NaN,NaN,NaN,NaN,NaN,2.2,3.20,3.00,2.25,3.25,3.25,2.3,3.3,2.90,2.30,3.2,3.20,2009-2010


In [3]:
save_file_name =  str('merge-1-' + league + '.csv')

## Standardize Team Names

In [4]:
epl_names_d = {'villa': 'aston-villa',
              'blackburn': 'blackburn-rovers',
              'birmingham': 'birmingham-city',
              'bolton': 'bolton-wanderers',
              'brighton': 'brighton-and-hove-albion',
              'brighton-&-Hove Albion': ' brighton-and-hove-albion',
              'cardiff': 'cardiff-city',
              'huddersfield': 'huddersfield-town',
              'hull': 'hull-city',
              'leicester': 'leicester-city',
              'leicester-cty': 'leicester-city',
              'man-city': 'manchester-city',
              'man-u': 'manchester-united',
              'man-utd': 'manchester-united',
              'manchester-utd': 'manchester-united',
              'man-united': 'manchester-united',
              'boro': 'middlesborough',
              'newcastle': 'newcastle-united',
              'newcastle-utd': 'newcastle-united',
              'norwich': 'norwich-city',
              'qpr': 'queens-park-rangers',
              'stoke': 'stoke-city',
              'swansea': 'swansea-city',
              'tottenham': 'tottenham-hotspur',
              'west-brom': 'west-bromwich-albion',
              'west-ham': 'west-ham-united',
              'wigan': 'wigan-athletic',
              'wolves': 'wolverhampton-wanderers'}

In [5]:
fn = 'english-premier-league_std_name_dict.pkl'

with open(REF_DATA_DIR / fn, 'wb') as handle:
    pickle.dump(epl_names_d, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

In [6]:
with open(REF_DATA_DIR / fn, 'rb') as handle:
    std_names_d = pickle.load(handle)

In [7]:
# Standardize the team names
df['h'] = df['h'].str.strip().str.lower().str.replace(' ', '-')
df.loc[df['h'].isin(std_names_d.keys()), 'h'] = df['h'].map(std_names_d)
df['a'] = df['a'].str.strip().str.lower().str.replace(' ', '-')
df.loc[df['a'].isin(std_names_d.keys()), 'a'] = df['a'].map(std_names_d)

In [8]:
df.head()

,a_corners,a_fouls,a_rcards,a_shots,a_sot,a_ycards,a,odds_awin_bet365,odds_draw_bet365,odds_hwin_bet365,odds_awin_BS,odds_draw_BS,odds_hwin_BS,odds_awin_BW,odds_draw_BW,odds_hwin_BW,n_Bb1X2,n_BbAsian,BbAsian_handicap,odds_ftgoalsu2.5_bbmean,odds_ftgoalso2.5_bbmean,odds_awin_bbmean,odds_asianaway_bbmean,odds_asianaway_bbmean.1,odds_draw_bbmean,odds_hwin_bbmean,odds_ftgoalsu2.5_bbmax,odds_ftgoalso2.5_bbmax,odds_awin_bbmax,odds_asianaway_bbmax,odds_asianhome_bbmax,odds_draw_bbmax,odds_hwin_bbmax,n_BbOU,date,a_ftgoals,h_ftgoals,odds_awin_GB,odds_draw_GB,odds_hwin_GB,h_corners,h_fouls,h_rcards,h_shots,h_sot,a_htgoals,h_htgoals,h_ycards,h,odds_awin_IW,odds_draw_IW,odds_hwin_IW,odds_awin_LB,odds_draw_LB,odds_hwin_LB,odds_awin_pinn,clodds_away_pinn,clodds_draw_pinn,clodds_hwin_pinn,odds_draw_pinn,odds_hwin_pinn,odds_awin_SB,odds_draw_SB,odds_hwin_SB,odds_awin_SJ,odds_draw_SJ,odds_hwin_SJ,odds_awin_VC,odds_draw_VC,odds_hwin_VC,odds_awin_WH,odds_draw_WH,odds_hwin_WH,season
0,6,14,0,14,7,2,wigan-athletic,5.50,3.60,1.67,5.5,3.5,1.62,5.00,3.40,1.65,37,21.0,0.0,1.81,1.92,5.33,3.99,1.22,3.51,1.66,1.91,2.04,6.05,4.40,1.28,3.77,1.72,32,2009-08-15,2,0,5.25,3.50,1.67,4,15,0,11,5,1,0,2,aston-villa,4.9,3.5,1.7,4.50,3.3,1.67,NaN,NaN,NaN,NaN,NaN,NaN,5.0,3.40,1.65,5.50,3.60,1.67,5.0,3.4,1.70,5.50,3.4,1.70,2009-2010
1,4,9,0,8,5,1,manchester-city,2.10,3.25,3.60,2.2,3.2,3.30,2.00,3.15,3.40,38,24.0,0.0,1.71,2.04,2.12,1.54,2.38,3.24,3.37,1.78,2.15,2.25,1.60,2.58,3.35,3.78,33,2009-08-15,2,0,2.10,3.25,3.40,5,12,0,17,9,1,0,2,blackburn-rovers,2.2,3.2,3.2,2.10,3.2,3.00,NaN,NaN,NaN,NaN,NaN,NaN,2.1,3.25,3.15,2.20,3.25,3.30,2.2,3.2,3.25,2.15,3.2,3.50,2009-2010
2,7,10,0,20,13,1,sunderland,3.25,3.25,2.25,3.3,3.2,2.20,3.15,3.15,2.15,38,23.0,0.0,1.66,2.12,3.15,2.23,1.61,3.20,2.24,1.70,2.25,3.40,2.33,1.68,3.31,2.37,33,2009-08-15,1,0,3.10,3.25,2.25,4,16,0,11,3,1,0,2,bolton-wanderers,3.2,3.2,2.2,3.00,3.2,2.10,NaN,NaN,NaN,NaN,NaN,NaN,2.9,3.20,2.25,3.10,3.25,2.30,3.1,3.2,2.25,3.20,3.2,2.30,2009-2010
3,4,15,0,7,3,2,hull-city,21.00,6.50,1.17,19.0,5.5,1.20,13.00,6.00,1.17,38,22.0,0.0,2.26,1.58,16.39,12.96,1.02,6.26,1.17,2.44,1.67,23.14,17.05,1.03,7.71,1.20,34,2009-08-15,1,2,15.00,6.00,1.20,12,13,0,26,12,1,1,1,chelsea,14.0,5.5,1.2,12.00,5.5,1.17,NaN,NaN,NaN,NaN,NaN,NaN,15.0,5.75,1.17,17.00,6.50,1.18,17.0,6.0,1.17,21.00,6.5,1.17,2009-2010
4,9,13,0,15,9,0,arsenal,2.30,3.25,3.20,2.2,3.2,3.25,2.25,3.15,2.95,38,24.0,0.0,1.70,2.06,2.28,1.63,2.20,3.21,3.07,1.77,2.20,2.42,1.73,2.27,3.34,3.34,34,2009-08-15,6,1,2.30,3.25,3.00,4,11,0,8,5,3,0,0,everton,2.3,3.1,3.1,2.25,3.0,2.88,NaN,NaN,NaN,NaN,NaN,NaN,2.2,3.20,3.00,2.25,3.25,3.25,2.3,3.3,2.90,2.30,3.2,3.20,2009-2010


In [9]:
df.set_index(['season', 'date'], inplace=True)
df.head()

a_corners  a_fouls  a_rcards  a_shots  a_sot  a_ycards                a  odds_awin_bet365  odds_draw_bet365  odds_hwin_bet365  odds_awin_BS  odds_draw_BS  odds_hwin_BS  odds_awin_BW  odds_draw_BW  odds_hwin_BW  n_Bb1X2  n_BbAsian  BbAsian_handicap  odds_ftgoalsu2.5_bbmean  odds_ftgoalso2.5_bbmean  odds_awin_bbmean  odds_asianaway_bbmean  odds_asianaway_bbmean.1  odds_draw_bbmean  odds_hwin_bbmean  odds_ftgoalsu2.5_bbmax  odds_ftgoalso2.5_bbmax  odds_awin_bbmax  odds_asianaway_bbmax  odds_asianhome_bbmax  odds_draw_bbmax  odds_hwin_bbmax  n_BbOU  a_ftgoals  h_ftgoals  odds_awin_GB  odds_draw_GB  odds_hwin_GB  h_corners  h_fouls  h_rcards  h_shots  h_sot  a_htgoals  h_htgoals  h_ycards                 h  odds_awin_IW  odds_draw_IW  odds_hwin_IW  odds_awin_LB  odds_draw_LB  odds_hwin_LB  odds_awin_pinn  clodds_away_pinn  clodds_draw_pinn  clodds_hwin_pinn  odds_draw_pinn  odds_hwin_pinn  odds_awin_SB  odds_draw_SB  odds_hwin_SB  odds_awin_SJ  odds_draw_SJ  \
season    date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
2009-2010 2009-08-15          6       14         0       14      7         2   wigan-athletic              5.50              3.60              1.67           5.5           3.5          1.62          5.00          3.40          1.65       37       21.0               0.0                     1.81                     1.92              5.33                   3.99                     1.22              3.51              1.66                    1.91                    2.04             6.05                  4.40                  1.28             3.77             1.72      32          2          0          5.25          3.50          1.67          4       15         0       11      5          1          0         2       aston-villa           4.9           3.5           1.7          4.50           3.3          1.67             NaN               NaN               NaN               NaN             NaN             NaN           5.0          3.40          1.65          5.50          3.60   
          2009-08-15          4        9         0        8      5         1  manchester-city              2.10              3.25              3.60           2.2           3.2          3.30          2.00          3.15          3.40       38       24.0               0.0                     1.71                     2.04              2.12                   1.54                     2.38              3.24              3.37                    1.78                    2.15             2.25                  1.60                  2.58             3.35             3.78      33          2          0          2.10          3.25          3.40          5       12         0       17      9          1          0         2  blackburn-rovers           2.2           3.2           3.2          2.10           3.2          3.00             NaN               NaN               NaN               NaN             NaN             NaN           2.1          3.25          3.15          2.20          3.25   
          2009-08-15          7       10         0    

In [10]:
### Name and Copy Heatmaps and Shotmaps

In [11]:
country = 'england'
league_dir = 'premier'

In [12]:
heatmap_fps = []
heatmap_dirs = [d for d in pathlib.Path(RAW_DATA_DIR /'soccer' / country / league_dir).rglob('**/*')
               if d.is_dir() and str(d).split('/')[-1] =='heatmaps']
for heatmap_dir in heatmap_dirs:
    heatmaps = [fp for fp in heatmap_dir.iterdir() if fp.is_file()]
    heatmap_fps.extend(heatmaps)

In [13]:
shotmap_fps = []
shotmap_dirs = [d for d in pathlib.Path(RAW_DATA_DIR /'soccer'/ country / league_dir).rglob('**/*')
               if d.is_dir() and str(d).split('/')[-1] =='shotmaps']
for shotmap_dir in shotmap_dirs:
    shotmaps = [fp for fp in shotmap_dir.iterdir() if fp.is_file()]
    shotmap_fps.extend(shotmaps)

In [14]:
fps = {}
for heatmap_fp in heatmap_fps:
#     print(heatmap_fp)
    h = str(heatmap_fp).split('__')[0].split('/')[-1].lower().replace(' ', '-')
    a = str(heatmap_fp).split('__')[1].split('/')[0].split('.')[0].lower().replace(' ', '-')
    if h in std_names_d.keys():
        h = std_names_d[h]
    if a in std_names_d.keys():
        a = std_names_d[a]
    season = str(heatmap_fp.parent.parent).split('/')[-1] 
    fps[season + '__' + 'date' + '__' + h + '__' + a] = heatmap_fp
for k in list(fps.keys())[0:5]:
    print(f'{k} : {fps[k]}')
#print(fps.items()[0:5])

2009-2010__date__arsenal__aston-villa : /media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/01-raw/soccer/england/premier/2009-2010/heatmaps/Arsenal__Aston Villa.png
2009-2010__date__arsenal__birmingham-city : /media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/01-raw/soccer/england/premier/2009-2010/heatmaps/Arsenal__Birmingham.png
2009-2010__date__arsenal__blackburn-rovers : /media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/01-raw/soccer/england/premier/2009-2010/heatmaps/Arsenal__Blackburn.png
2009-2010__date__arsenal__bolton-wanderers : /media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/01-raw/soccer/england/premier/2009-2010/heatmaps/Arsenal__Bolton.png
2009-2010__date__arsenal__burnley : /media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/01-raw/soccer/england/premier/2009-2010/heatmaps/Arsenal__Burnley.png


In [15]:
fpss = {}
for shotmap_fp in shotmap_fps:
#     print(heatmap_fp)
    h = str(shotmap_fp).split('__')[0].split('/')[-1].lower().replace(' ', '-')
    a = str(shotmap_fp).split('__')[1].split('/')[0].split('.')[0].lower().replace(' ', '-')
    if h in std_names_d.keys():
        h = std_names_d[h]
    if a in std_names_d.keys():
        a = std_names_d[a]
    season = str(shotmap_fp.parent.parent).split('/')[-1] 
    fpss[season + '__' + 'date' + '__' + h + '__' + a] = shotmap_fp

    # Take a peek at the original shotmap filepaths 
for k in list(fpss.keys())[0:5]:
    print(f'{k} : {fpss[k]}')

2009-2010__date__arsenal__aston-villa : /media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/01-raw/soccer/england/premier/2009-2010/shotmaps/Arsenal__Aston Villa.png
2009-2010__date__arsenal__birmingham-city : /media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/01-raw/soccer/england/premier/2009-2010/shotmaps/Arsenal__Birmingham.png
2009-2010__date__arsenal__blackburn-rovers : /media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/01-raw/soccer/england/premier/2009-2010/shotmaps/Arsenal__Blackburn.png
2009-2010__date__arsenal__bolton-wanderers : /media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/01-raw/soccer/england/premier/2009-2010/shotmaps/Arsenal__Bolton.png
2009-2010__date__arsenal__burnley : /media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/01-raw/soccer/england/premier/2009-2010/shotmaps/Arsenal__Burnley.png


In [16]:
df1 = df.copy(deep=True)

def copy_src_to_dest(src_fp, dest_fp):
    if not dest_fp.exists():
        dest_fp.parent.mkdir(parents=True, exist_ok=True)
        # Copy the original files without touching them
        dest_fp.write_bytes(src_fp.read_bytes())

def form_image_fp(row, league=None, image_type=None):
    row = row.copy(deep=True)
    season = row.name[0]
    date = str(row.name[1].date())
    h = row['h']
    a = row['a']
    key = season + '__' + 'date' + '__' + h + '__' + a
    src_fp = fps[key]
    ext = src_fp.suffix
    dest_fp = INTERIM_DATA_DIR / league/ 'heatmaps' / str(key.replace('date', date) + src_fp.suffix) 
    copy_src_to_dest(src_fp, dest_fp)
    rel_path = str(dest_fp).replace(str(PROJECT_DIR), '')
    return rel_path
    
df1.loc[:, 'heatmap_path'] = df1.apply(form_image_fp,
                                       league=league,
                                       image_type='heatmaps', axis=1)
df1.head()


a_corners  a_fouls  a_rcards  a_shots  a_sot  a_ycards                a  odds_awin_bet365  odds_draw_bet365  odds_hwin_bet365  odds_awin_BS  odds_draw_BS  odds_hwin_BS  odds_awin_BW  odds_draw_BW  odds_hwin_BW  n_Bb1X2  n_BbAsian  BbAsian_handicap  odds_ftgoalsu2.5_bbmean  odds_ftgoalso2.5_bbmean  odds_awin_bbmean  odds_asianaway_bbmean  odds_asianaway_bbmean.1  odds_draw_bbmean  odds_hwin_bbmean  odds_ftgoalsu2.5_bbmax  odds_ftgoalso2.5_bbmax  odds_awin_bbmax  odds_asianaway_bbmax  odds_asianhome_bbmax  odds_draw_bbmax  odds_hwin_bbmax  n_BbOU  a_ftgoals  h_ftgoals  odds_awin_GB  odds_draw_GB  odds_hwin_GB  h_corners  h_fouls  h_rcards  h_shots  h_sot  a_htgoals  h_htgoals  h_ycards                 h  odds_awin_IW  odds_draw_IW  odds_hwin_IW  odds_awin_LB  odds_draw_LB  odds_hwin_LB  odds_awin_pinn  clodds_away_pinn  clodds_draw_pinn  clodds_hwin_pinn  odds_draw_pinn  odds_hwin_pinn  odds_awin_SB  odds_draw_SB  odds_hwin_SB  odds_awin_SJ  odds_draw_SJ  \
season    date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
2009-2010 2009-08-15          6       14         0       14      7         2   wigan-athletic              5.50              3.60              1.67           5.5           3.5          1.62          5.00          3.40          1.65       37       21.0               0.0                     1.81                     1.92              5.33                   3.99                     1.22              3.51              1.66                    1.91                    2.04             6.05                  4.40                  1.28             3.77             1.72      32          2          0          5.25          3.50          1.67          4       15         0       11      5          1          0         2       aston-villa           4.9           3.5           1.7          4.50           3.3          1.67             NaN               NaN               NaN               NaN             NaN             NaN           5.0          3.40          1.65          5.50          3.60   
          2009-08-15          4        9         0        8      5         1  manchester-city              2.10              3.25              3.60           2.2           3.2          3.30          2.00          3.15          3.40       38       24.0               0.0                     1.71                     2.04              2.12                   1.54                     2.38              3.24              3.37                    1.78                    2.15             2.25                  1.60                  2.58             3.35             3.78      33          2          0          2.10          3.25          3.40          5       12         0       17      9          1          0         2  blackburn-rovers           2.2           3.2           3.2          2.10           3.2          3.00             NaN               NaN               NaN               NaN             NaN             NaN           2.1          3.25          3.15          2.20          3.25   
          2009-08-15          7       10         0    

In [17]:
def form_image_fp(row, league=None, image_type=None):
    row = row.copy(deep=True)
    season = row.name[0]
    date = str(row.name[1].date())
    h = row['h']
    a = row['a']
    key = season + '__' + 'date' + '__' + h + '__' + a
    src_fp = fpss[key]
    ext = src_fp.suffix
    dest_fp = INTERIM_DATA_DIR / league/ 'shotmaps' / str(key.replace('date', date) + src_fp.suffix) 
    copy_src_to_dest(src_fp, dest_fp)
    rel_path = str(dest_fp).replace(str(PROJECT_DIR), '')
    return rel_path
    
df1.loc[:, 'shotmap_path'] = df1.apply(form_image_fp,
                                       league=league,
                                       image_type='shotmaps', axis=1)
df1.head()


a_corners  a_fouls  a_rcards  a_shots  a_sot  a_ycards                a  odds_awin_bet365  odds_draw_bet365  odds_hwin_bet365  odds_awin_BS  odds_draw_BS  odds_hwin_BS  odds_awin_BW  odds_draw_BW  odds_hwin_BW  n_Bb1X2  n_BbAsian  BbAsian_handicap  odds_ftgoalsu2.5_bbmean  odds_ftgoalso2.5_bbmean  odds_awin_bbmean  odds_asianaway_bbmean  odds_asianaway_bbmean.1  odds_draw_bbmean  odds_hwin_bbmean  odds_ftgoalsu2.5_bbmax  odds_ftgoalso2.5_bbmax  odds_awin_bbmax  odds_asianaway_bbmax  odds_asianhome_bbmax  odds_draw_bbmax  odds_hwin_bbmax  n_BbOU  a_ftgoals  h_ftgoals  odds_awin_GB  odds_draw_GB  odds_hwin_GB  h_corners  h_fouls  h_rcards  h_shots  h_sot  a_htgoals  h_htgoals  h_ycards                 h  odds_awin_IW  odds_draw_IW  odds_hwin_IW  odds_awin_LB  odds_draw_LB  odds_hwin_LB  odds_awin_pinn  clodds_away_pinn  clodds_draw_pinn  clodds_hwin_pinn  odds_draw_pinn  odds_hwin_pinn  odds_awin_SB  odds_draw_SB  odds_hwin_SB  odds_awin_SJ  odds_draw_SJ  \
season    date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
2009-2010 2009-08-15          6       14         0       14      7         2   wigan-athletic              5.50              3.60              1.67           5.5           3.5          1.62          5.00          3.40          1.65       37       21.0               0.0                     1.81                     1.92              5.33                   3.99                     1.22              3.51              1.66                    1.91                    2.04             6.05                  4.40                  1.28             3.77             1.72      32          2          0          5.25          3.50          1.67          4       15         0       11      5          1          0         2       aston-villa           4.9           3.5           1.7          4.50           3.3          1.67             NaN               NaN               NaN               NaN             NaN             NaN           5.0          3.40          1.65          5.50          3.60   
          2009-08-15          4        9         0        8      5         1  manchester-city              2.10              3.25              3.60           2.2           3.2          3.30          2.00          3.15          3.40       38       24.0               0.0                     1.71                     2.04              2.12                   1.54                     2.38              3.24              3.37                    1.78                    2.15             2.25                  1.60                  2.58             3.35             3.78      33          2          0          2.10          3.25          3.40          5       12         0       17      9          1          0         2  blackburn-rovers           2.2           3.2           3.2          2.10           3.2          3.00             NaN               NaN               NaN               NaN             NaN             NaN           2.1          3.25          3.15          2.20          3.25   
          2009-08-15          7       10         0    

In [18]:
df1.iloc[0].loc['heatmap_path']

'/data/02-interim/football-data/english-premier-league/heatmaps/2009-2010__2009-08-15__aston-villa__wigan-athletic.png'

In [19]:
df1.iloc[0].loc['shotmap_path']

'/data/02-interim/football-data/english-premier-league/shotmaps/2009-2010__2009-08-15__aston-villa__wigan-athletic.png'

In [20]:
df1.to_csv(INTERIM_DATA_DIR / league / save_file_name, index=True)

In [21]:
## Tidy up code - leave image paths as relative to project dir